In [1]:
import numpy as np
import tensorflow as tf
import sklearn.preprocessing as skp

In [2]:
text_file_path = "data/c50-articles.txt"
label_file_path = "data/c50-labels.txt"

In [3]:
text_tokenizer =  tf.keras.preprocessing.text.Tokenizer(num_words=1000)

with open(text_file_path) as text_file:
    text_tokenizer.fit_on_texts(text_file)
    
with open(text_file_path) as text_file:
    integer_text_sequences = text_tokenizer.texts_to_sequences(text_file)

len(integer_text_sequences)

2500

In [4]:
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
     integer_text_sequences, maxlen=50, padding='post', truncating='post')

padded_sequences.shape

(2500, 50)

In [5]:
label_tokenizer =  tf.keras.preprocessing.text.Tokenizer(lower=False)

with open(label_file_path) as label_file:
    label_tokenizer.fit_on_texts(label_file)

with open(label_file_path) as label_file:
    label_sequences = label_tokenizer.texts_to_sequences(label_file)

In [6]:
def convert_label_integer_sequence_to_onehot_rep(label_sequence):
    return np.eye(len(label_tokenizer.word_index), k=label_sequence[0])[0]

one_hot_labels = list(map(convert_label_integer_sequence_to_onehot_rep, label_sequences))